In [1]:
import os
import time
import json
import re
import pprint
import csv
import time
import sys
import xml.etree.ElementTree as ET
from pathlib import Path

In [2]:
# regex objects
proxy_re = re.compile('proxy\.bc\.edu\/login\?url=', re.IGNORECASE)
remove_protocol_re = re.compile('https?:\/\/', re.IGNORECASE)
domain_name_re = re.compile('(www\.)?(?P<dn>[a-zA-Z0-9.\-_]*).*', re.IGNORECASE)

In [3]:
# files to be read by this script
db_dump_file = 'export_1521748808.xml'
databases_file_name = 'all_databases.txt'
databases_domain_names_file_name = 'all_databases_domain_names.txt'

In [4]:
# load xml dump and get root element

# get filename without extension
db_dump_file_no_extention = re.sub('\.xml', '', db_dump_file)

my_dump_file = Path(db_dump_file)
if my_dump_file.is_file():
    tree = ET.parse(db_dump_file)
    root = tree.getroot()
else:
    sys.exit("Can't find %s" % db_dump_file)

In [5]:
# test that import worked
root

<Element 'libguides' at 0x10460f3b8>

In [6]:
# get total number of accounts
total_accounts = len(root.findall('./accounts/account'))
total_accounts

99

In [7]:
# get total number of guides
total_guides = len(root.findall('./guides/guide'))
total_guides

815

In [8]:
# get total number of pages
total_pages = len(root.findall('.//pages/page'))
total_pages

4655

In [9]:
# get total number of boxes
total_boxes = len(root.findall('.//boxes/box'))
total_boxes

13361

In [10]:
# get total number of assets
total_assets = len(root.findall('.//assets/asset'))
total_assets

29980

In [11]:
# get total number of link assets
link_count = 0
for link in root.findall('.//assets/asset'):
    if link.findtext('type') == "Link":
        link_count += 1
        
link_count

14656

In [12]:
# get all guide authors and add them to accounts_dict dictionary
accounts_count = 0
accounts_dict = {}
for account in root.findall('./accounts/account'):
    acct_id = account.findtext('id')
    acct_fname = account.findtext('first_name')
    acct_lname = account.findtext('last_name')
    accounts_dict[acct_id] = "%s %s" % (acct_fname, acct_lname)
    accounts_count += 1
    
accounts_count

99

In [22]:
# test output
accounts_dict['10678']

'Leslie Homzie'

In [33]:
# read in list of databases
databases_file_name = 'all_databases.txt'
my_db_file = Path(databases_file_name)
db_names_set = []
if my_db_file.is_file():
    with open(my_db_file, 'r') as f:
        data = json.load(f)

    # pull out key 'databases' from file
    db_names_set = data['databases']
else:
    sys.exit("Can't find %s" % file_name)

In [34]:
len(db_names_set)

737

In [35]:
# read in list of database domain names
my_db_file = Path(databases_domain_names_file_name)
db_domain_names_set = []
if my_db_file.is_file():
    with open(my_db_file, 'r') as f:
        data = json.load(f)

    # pull out key 'databases' from file
    db_domain_names_set = data['databases']
else:
    sys.exit("Can't find %s" % file_name)

In [36]:
len(db_domain_names_set)

336

In [37]:
# parse guides
guides_parsed_count = 0
pages_parsed_count = 0
boxes_parsed_count = 0
assets_parsed_count = 0
link_assets_parsed_count = 0

proxied_asset_count = 0
proxied_assets_list = []

suspicious_asset_count = 0
suspicious_asset_list = []

# parse guides
for guide in root.findall('./guides/guide'):
    guide_id = guide.findtext('id')
    guide_name = guide.findtext('name')
    guide_url = guide.findtext('url')
    guide_owner = guide.findtext('owner/id')
    guide_status = guide.findtext('status')
    
    # parse pages in a guide
    local_pages_count = 0
    pages_dict = {}
    for page in guide.findall('./pages/page'):
        page_id = page.findtext('id')
        page_name = page.findtext('name')
        page_url = page.findtext('url')
        
        # parse boxes in a page
        local_boxes_count = 0
        boxes_dict = {}
        for box in page.findall('./boxes/box'):
            box_id = box.findtext('id')
            box_name = box.findtext('name')
            
            # construct box_url
            # box_url = "%s#s-lg-box-%s" % (page_url, box_id)
            box_url = 'https://bc.libapps.com/libguides/admin_c.php?g=%s&p=%s#s-lg-box-%s' %(guide_id, page_id, box_id)
            
            # parse assets in a box
            local_assets_count = 0
            assets_dict = {}
            for asset in box.findall('./assets/asset'):
                asset_id = asset.findtext('id')
                asset_name = asset.findtext('name')
                asset_type = asset.findtext('type')
                
                # check if this asset is of type "Link"
                if asset_type == "Link":
                    asset_url_raw = asset.findtext('url')
                    
                    if not asset_url_raw:
                        # TODO log error here
                        continue
                    
                    # shift to lowercase 
                    asset_url_lower = asset_url_raw.lower()
                    
                    # strip out protocol (http:// | https://)
                    asset_url_clean = re.sub(remove_protocol_re, "", asset_url_lower)
                    
                    # strip out trailing forward slash
                    asset_url_clean = asset_url_clean.rstrip('/')
                    
                    # first, check if this has the proxy prefix
                    has_proxy = proxy_re.match(asset_url_clean)
                    if has_proxy:
                        asset_url = re.sub(proxy_re, '', asset_url_clean)
                        
                        # add to proxied_assets_list
                        proxied_assets_list.append(
                            { 
                                "guide_owner": accounts_dict[guide_owner],
                                "asset_name": asset_name,
                                "reason": "uses proxy prefix",
                                "box_url": box_url,
                                "guide_status": guide_status,
                                "guide_id": guide_id,
                                "guide_name": guide_name,
                                # "guide_url": guide_url,
                                # "page_id": page_id,
                                "page_name": page_name,
                                # "page_url": page_url,
                                # "box_id": box_id,
                                "box_name": box_name,
                                # "asset_id": asset_id,
                                "asset_url_raw": asset_url_raw,
                                "matches_on_known_db_name": asset_url
                            }
                        )
                        proxied_asset_count += 1
                    else:
                        asset_url = asset_url_clean
                        
                    
                    # next, check if this url matches the full database url
                    # if not, then check to see if the asset_url matches a db domain name
                    if asset_url in db_names_set:
                        suspicious_asset_list.append(
                            {
                                
                                "guide_owner": accounts_dict[guide_owner],
                                "asset_name": asset_name,
                                "reason": "matches full database url",
                                "box_url": box_url,
                                "guide_status": guide_status,
                                "guide_id": guide_id,
                                "guide_name": guide_name,
                                # "guide_url": guide_url,
                                # "page_id": page_id,
                                "page_name": page_name,
                                # "page_url": page_url,
                                # "box_id": box_id,
                                "box_name": box_name,
                                # "asset_id": asset_id,
                                "asset_url_raw": asset_url_raw,
                                "matches_on_known_db_name": asset_url
                            }
                        )
                        suspicious_asset_count += 1
                    else: 
                        # extract out the domain name from the asset_url variable
                        get_dn = domain_name_re.search(asset_url)
                        if get_dn:
                            matched_dn = get_dn.group("dn")
                            if matched_dn:
                                asset_url_dn = matched_dn
                            else:
                                # TODO log an error here
                                continue

                        # match asset_url_dn to known database dn's
                        if asset_url_dn in db_domain_names_set:
                            # add to suspicious_asset_list
                            suspicious_asset_list.append(
                                {
                                    "guide_owner": accounts_dict[guide_owner],
                                    "asset_name": asset_name,
                                    "reason": "matches a database domain name",
                                    "box_url": box_url,
                                    "guide_status": guide_status,
                                    "guide_id": guide_id,
                                    "guide_name": guide_name,
                                    # "guide_url": guide_url,
                                    # "page_id": page_id,
                                    "page_name": page_name,
                                    # "page_url": page_url,
                                    # "box_id": box_id,
                                    "box_name": box_name,
                                    # "asset_id": asset_id,
                                    "asset_url_raw": asset_url_raw,
                                    "matches_on_known_db_name": asset_url_dn
                                }
                            )
                            suspicious_asset_count += 1

                    link_assets_parsed_count += 1
                    # END check if this asset is of type "Link"
                
                local_assets_count += 1
                assets_parsed_count += 1
                # END parse assets in a box
            
            local_boxes_count += 1
            boxes_parsed_count += 1
            # END parse boxes in a page
            
        local_pages_count += 1
        pages_parsed_count += 1
        # END parse pages in a guide
        
    guides_parsed_count += 1
    # END parse guides

In [38]:
print("guides_parsed_count: %s" % guides_parsed_count)
print("pages_parsed_count: %s" % pages_parsed_count)
print("boxes_parsed_count: %s" % boxes_parsed_count)
print("assets_parsed_count: %s" % assets_parsed_count)
print("link_assets_parsed_count: %s" % link_assets_parsed_count)
print("proxied_asset_count: %s" % proxied_asset_count)
print("suspicious_asset_count: %s" % suspicious_asset_count)

guides_parsed_count: 815
pages_parsed_count: 4655
boxes_parsed_count: 13361
assets_parsed_count: 29394
link_assets_parsed_count: 14070
proxied_asset_count: 1014
suspicious_asset_count: 2091


In [39]:
#pprint.pprint(proxied_assets_list)

In [40]:
#pprint.pprint(suspicious_asset_list)

In [41]:
# convert suspicious_asset_list to a csv file
# https://stackoverflow.com/questions/3086973/how-do-i-convert-this-list-of-dictionaries-to-a-csv-file
fieldnames = [
    "guide_id", "guide_name", "guide_url", "guide_owner", 
    "page_id", "page_name", "page_url", 
    "box_id", "box_name", "box_url", 
    "asset_id", "asset_name", "asset_url_raw", "asset_url_to_match"
]
timestr = time.strftime("%Y%m%d-%H%M%S")
csv_filename = 'suspicious_assets-%s-%s.csv' % (db_dump_file_no_extention, timestr)
keys = suspicious_asset_list[0].keys()
with open(csv_filename, 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(suspicious_asset_list)